In [10]:
import re
import json

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType

if 'spark' in locals() or 'spark' in globals():
    spark.stop()

spark = SparkSession\
    .builder\
    .appName("Criando tabelas bronze de pokemons")\
    .getOrCreate()

spark

In [2]:
df = spark.read.json("s3a://datalake/raw/pokemon/*.json")

In [3]:
schema_str = df.schema.json()

# Remove all hyphens from field name in the schema and substitute for underscores
new_schema_str = re.sub("\-+", "_", schema_str)

new_schema_json = json.loads(new_schema_str)
new_schema_json

df = spark.createDataFrame(df.collect(), StructType.fromJson(new_schema_json))

In [4]:
spark.sql("CREATE DATABASE IF NOT EXISTS pokeapi")
spark.sql("SHOW DATABASES").show()

Hive Session ID = d1e13d9f-5612-43bf-b9d0-360dfea9a842


+-------------+
|    namespace|
+-------------+
|      default|
|    deltabase|
|dundermifflin|
|      pokeapi|
+-------------+



In [5]:
spark.sql("DROP TABLE IF EXISTS pokeapi.tbl_bronze_pokemons")
df.write\
    .option("path", "s3a://datalake/bronze/pokemons/")\
    .format('parquet')\
    .mode('overwrite')\
    .saveAsTable('pokeapi.tbl_bronze_pokemons')

23/04/03 18:57:52 WARN TaskSetManager: Stage 4 contains a task of very large size (7392 KiB). The maximum recommended task size is 1000 KiB.


In [6]:
df2 = spark.read.json("s3a://datalake/raw/evolution/*.json")

In [7]:
spark.sql("DROP TABLE IF EXISTS pokeapi.tbl_bronze_evolutions")
df2.write\
    .option("path", "s3a://datalake/bronze/evolution/")\
    .format('parquet')\
    .mode('overwrite')\
    .saveAsTable('pokeapi.tbl_bronze_evolutions')

In [8]:
spark.sql("SHOW TABLES IN pokeapi").show(truncate = False)

+---------+---------------------+-----------+
|namespace|tableName            |isTemporary|
+---------+---------------------+-----------+
|pokeapi  |tbl_bronze_evolutions|false      |
|pokeapi  |tbl_bronze_pokemons  |false      |
+---------+---------------------+-----------+



In [9]:
spark.stop()